## Day 5: Supply Stacks

[![nbviewer](https://raw.githubusercontent.com/jupyter/design/master/logos/Badges/nbviewer_badge.svg)](https://nbviewer.org/github/mazharenko/AoC-2022/tree/HEAD/notebooks/day05/puzzle.ipynb)

### Parsing

The format of the drawing of the starting stacks is rather peculiar and it can be struggling to find the best way to parse it. 


In [52]:
#!value --name sampleRaw --from-file ./data_sample.txt

In [53]:

#!share sampleRaw --from value
sampleRaw

    [D]    
[N] [C]    
[Z] [M] [P]
 1   2   3 

move 1 from 2 to 1
move 3 from 1 to 3
move 2 from 2 to 1
move 1 from 1 to 2


First, we can split the two parts making use of the previously defined `Pattern2.read` function and the nominal item parsing funcion.

In [54]:
#!share sampleRaw --from value
#load "../common/common.fsx"

sampleRaw |> Pattern2.read id |> Array.head

    [D]    
[N] [C]    
[Z] [M] [P]
 1   2   3 

The `Pattern2.read` function effectively produces a jagged array, which can then be iterated upside down

In [55]:
let linearizeStacksInput verbose input = 
    let jagged = 
        input |> Pattern1.read Seq.toArray

    if (verbose) then display jagged |> ignore

    [|
        for j in 0..(Array.length jagged[0]-1) do
            for i in (Array.length jagged - 1)..(-1)..0 do
                yield jagged[i][j]
    |] |> String

sampleRaw
|> Pattern2.read id 
|> Array.head
|> linearizeStacksInput true

index,value
0,"[ , , , , [, D, ], , , , ]"
1,"[ [, N, ], , [, C, ], , , , ]"
2,"[ [, Z, ], , [, M, ], , [, P, ] ]"
3,"[ , 1, , , , 2, , , , 3, ]"


 [[ 1ZN  ]]      [[[2MCD ]]]     [  3P   ]  

The string generated this way can then simply be read with Farkle. For the types, it is reasonable to represent a stack as a LIFO `List` and make sure it is filled up in reversed order, so that for sequence `2MCD` `D` is the list head.

In [56]:
type Crates = char list
type Stacks = Map<int,Crates>

In [57]:
#r "nuget:Farkle, 6.5.0"

Installed Packages Farkle, 6.5.0

In [58]:
open Farkle
open Farkle.Builder
open Farkle.Builder.Regex


let private crates = 
    chars PredefinedSets.AllLetters 
    |> atLeast 1 
    |> terminal "Crates" (T(fun _ x -> x.ToArray()))
let private number = Terminals.int "Number"
let private stack = "Stack" ||= [
    !@ number .>>. crates => fun stackId crates -> (stackId, crates |> Array.rev |> List.ofArray)
]
let stacks = nonterminal "Stacks"
stacks.SetProductions(
    !@ stack .>>. stacks => (fun (id, crates) xs -> Map.add id crates xs),
    !@ stack => fun (id, crates) -> Map.empty |> Map.add id crates
)

let private parser = 
    stacks 
    |> DesigntimeFarkle.addNoiseSymbol "[" (char '[')
    |> DesigntimeFarkle.addNoiseSymbol "]" (char ']')
    |> RuntimeFarkle.build
let parseStacks s : Stacks = 
    RuntimeFarkle.parseString parser s 
    |> Result.get

parseStacks "[[ 1ZN  ]]      [[[2MCD ]]]     [  3P   ]  "

key,value
1,"[ N, Z ]"
2,"[ D, C, M ]"
3,[ P ]


The second part, the one with the rearrangement procedure, is trivial for Farkle

In [59]:
type RearrangementStep = {Count: int; From: int; To: int}

let private number = Terminals.int "Number"
let private step = "Step" ||= [
    !& "move" .>>. number .>> "from" .>>. number .>> "to" .>>. number
        => fun count from to' -> {Count = count; From = from; To = to'}
]
let private parser = RuntimeFarkle.build step
let private parseStep s : RearrangementStep = 
    RuntimeFarkle.parseString parser s 
    |> Result.get

let parseRearrangement s = 
    Pattern1.read parseStep s |> Array.toList

sampleRaw
|> Pattern2.read id 
|> Array.last
|> displayPipe
|> parseRearrangement


move 1 from 2 to 1
move 3 from 1 to 3
move 2 from 2 to 1
move 1 from 1 to 2


index,Count,From,To
0,1,2,1
1,3,1,3
2,2,2,1
3,1,1,2


Now, all these can be built up together in a single parsing function:

In [60]:
let parse input = 
    let inputSplit = Pattern2.read id input
    let stacksInput = inputSplit[0]
    let rearrangementInput = inputSplit[1]
    stacksInput |> linearizeStacksInput false |> parseStacks, parseRearrangement rearrangementInput

let (sampleStacks, sampleRearrangement) = parse sampleRaw
display sampleStacks
display sampleRearrangement

key,value
1,"[ N, Z ]"
2,"[ D, C, M ]"
3,[ P ]


index,Count,From,To
0,1,2,1
1,3,1,3
2,2,2,1
3,1,1,2


In [61]:
#!value --name actualRaw --from-file ./data_actual.txt

In [62]:
#!share actualRaw --from value

let (actualStacks, actualRearrangement) = parse actualRaw

display actualStacks
display actualRearrangement

key,value
1,"[ R, Q, G, P, C, F ]"
2,"[ P, C, T, W ]"
3,"[ C, M, P, H, B ]"
4,"[ R, P, M, S, Q, T, L ]"
5,"[ N, G, V, Z, J, H, P ]"
6,"[ J, P, D ]"
7,"[ R, T, J, F, Z, P, G, L ]"
8,"[ J, T, P, F, C, H, L, N ]"
9,"[ W, C, T, H, Q, Z, V, G ]"


index,Count,From,To
0,2,2,8
1,2,1,6
2,8,7,1
3,7,5,4
4,1,6,4
5,1,6,3
6,6,3,5
7,9,8,1
8,3,6,7
9,14,4,1


### Part 1

Since `Stacks` type is immutable, each rearrangement step should produce changed `Stacks`. First, the stack under the specified key must be split in two. The second part then replaces the source stack list, and the first part then prepends the target stack list. To conform with the "one at a time" condition, we can reverse it first.

In [63]:
module Part1 = 
    let execute (step : RearrangementStep) (stacks : Stacks) : Stacks = 
        let (removed, left) = List.splitAt step.Count stacks[step.From]
        stacks
        |> Map.change step.From (fun _ -> Some left)
        |> Map.change step.To (
            function 
            | None -> Some left 
            | Some toExisting -> Some ((List.rev removed) @ toExisting)
        )

Using the `execute` function, a rearrangement step list can be recursively applied to the initial state of Stacks, producing the target state.

In [64]:
let rec rearrange verbose (executeStep : RearrangementStep->Stacks->Stacks) (steps : RearrangementStep list) (stacks : Stacks) : Stacks = 
    match steps with
    | [] -> stacks
    | step::rest -> 
        if (verbose) then stacks |> display |> ignore
        if (verbose) then step |> display |> ignore
        let newStacks = executeStep step stacks
        rearrange verbose executeStep rest newStacks


In [65]:
let topCrates (stacks : Stacks) = 
    stacks
    |> Map.toSeq
    |> Seq.sortBy fst
    |> Seq.map (snd >> List.head)
    |> String.Concat

rearrange true Part1.execute sampleRearrangement sampleStacks
|> displayPipe
|> topCrates

key,value
1,"[ N, Z ]"
2,"[ D, C, M ]"
3,[ P ]


Count,From,To
1,2,1


key,value
1,"[ D, N, Z ]"
2,"[ C, M ]"
3,[ P ]


Count,From,To
3,1,3


key,value
1,[ ]
2,"[ C, M ]"
3,"[ Z, N, D, P ]"


Count,From,To
2,2,1


key,value
1,"[ M, C ]"
2,[ ]
3,"[ Z, N, D, P ]"


Count,From,To
1,1,2


key,value
1,[ C ]
2,[ M ]
3,"[ Z, N, D, P ]"


CMZ

For the actual input:

In [66]:
actualStacks |> display

rearrange false Part1.execute actualRearrangement actualStacks
|> displayPipe
|> topCrates

key,value
1,"[ R, Q, G, P, C, F ]"
2,"[ P, C, T, W ]"
3,"[ C, M, P, H, B ]"
4,"[ R, P, M, S, Q, T, L ]"
5,"[ N, G, V, Z, J, H, P ]"
6,"[ J, P, D ]"
7,"[ R, T, J, F, Z, P, G, L ]"
8,"[ J, T, P, F, C, H, L, N ]"
9,"[ W, C, T, H, Q, Z, V, G ]"


key,value
1,"[ D, Q ]"
2,"[ H, M, H, F, P, W, R, P, T, P, C, T ]"
3,"[ B, C ]"
4,"[ J, T ]"
5,"[ Q, F, V, P, G, G, F, N, J, J, H, L, R, Z, H, G, T ]"
6,[ J ]
7,"[ C, V, Z, Q ]"
8,"[ C, P, N, G, M, L, L, C, Z, T, P, P, P ]"
9,"[ W, R, S ]"


DHBJQJCCW

### Part 2

Crates are moved just like for Part 1, save for the List reversal.

In [67]:
module Part2 = 
    let execute (step : RearrangementStep) (stacks : Stacks) : Stacks = 
        let (removed, left) = List.splitAt step.Count stacks[step.From]
        stacks
        |> Map.change step.From (fun _ -> Some left)
        |> Map.change step.To (
            function 
            | None -> Some left 
            | Some toExisting -> Some (removed @ toExisting)
        )

In [68]:

rearrange true Part2.execute sampleRearrangement sampleStacks
|> displayPipe
|> topCrates

key,value
1,"[ N, Z ]"
2,"[ D, C, M ]"
3,[ P ]


Count,From,To
1,2,1


key,value
1,"[ D, N, Z ]"
2,"[ C, M ]"
3,[ P ]


Count,From,To
3,1,3


key,value
1,[ ]
2,"[ C, M ]"
3,"[ D, N, Z, P ]"


Count,From,To
2,2,1


key,value
1,"[ C, M ]"
2,[ ]
3,"[ D, N, Z, P ]"


Count,From,To
1,1,2


key,value
1,[ M ]
2,[ C ]
3,"[ D, N, Z, P ]"


MCD

For the actual input:

In [69]:
actualStacks |> display

rearrange false Part2.execute actualRearrangement actualStacks
|> displayPipe
|> topCrates

key,value
1,"[ R, Q, G, P, C, F ]"
2,"[ P, C, T, W ]"
3,"[ C, M, P, H, B ]"
4,"[ R, P, M, S, Q, T, L ]"
5,"[ N, G, V, Z, J, H, P ]"
6,"[ J, P, D ]"
7,"[ R, T, J, F, Z, P, G, L ]"
8,"[ J, T, P, F, C, H, L, N ]"
9,"[ W, C, T, H, Q, Z, V, G ]"


key,value
1,"[ W, G ]"
2,"[ J, P, P, W, C, C, M, P, N, C, Z, P ]"
3,"[ V, G ]"
4,"[ R, H ]"
5,"[ L, M, T, R, N, H, Z, T, P, D, F, Z, C, F, R, P, G ]"
6,[ S ]
7,"[ J, F, L, P ]"
8,"[ J, B, J, H, T, V, P, Q, H, C, Q, L, T ]"
9,"[ T, G, Q ]"


WJVRLSJJT